#### **IMPORTANDO AS BIBLIOTECAS**

In [33]:
# Bibliotecas para tratar os dados
import pandas as pd

# Bibliotecas para EDA
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as py
import folium

# Configurar para não exibir warnings
import warnings
warnings.filterwarnings('ignore')

# Configurar para exibir todas as colunas de um dataframe do Pandas
pd.set_option('display.max_columns',None)


#### **IMPORTANDO O DATASET**

In [34]:
airbnb = pd.read_csv('train.csv')
airbnb.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [35]:
print(f'O dataframe possui {airbnb.shape[0]} linhas e {airbnb.shape[1]} colunas.')

O dataframe possui 48895 linhas e 16 colunas.


In [36]:
airbnb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              48895 non-null  int64  
 1   name                            48879 non-null  object 
 2   host_id                         48895 non-null  int64  
 3   host_name                       48874 non-null  object 
 4   neighbourhood_group             48895 non-null  object 
 5   neighbourhood                   48895 non-null  object 
 6   latitude                        48895 non-null  float64
 7   longitude                       48895 non-null  float64
 8   room_type                       48895 non-null  object 
 9   price                           48895 non-null  int64  
 10  minimum_nights                  48895 non-null  int64  
 11  number_of_reviews               48895 non-null  int64  
 12  last_review                     

Temos alguns valores nulos nas colunas name, host_name, last_review e review_per_month. 

In [37]:
numericas = airbnb.select_dtypes(include='int64')
nao_numericas = airbnb.select_dtypes(exclude='int64')

print(f'Temos {numericas.shape[1]} colunas numéricas e {nao_numericas.shape[1]} colunas não-numéricas.')

Temos 7 colunas numéricas e 9 colunas não-numéricas.


#### **EXPLORANDO OS DADOS**
Primeiramente, iremos avaliar as distribuições dos tipos de acomodações.

In [38]:
fig = px.histogram(airbnb, x='room_type', color='room_type',text_auto='.0f')

fig.update_traces(textposition='outside')

fig.update_layout(title='Distribuição dos Tipos de Acomodações',
                  xaxis_title='Tipo',
                  yaxis_title='Quantidade',
                  showlegend=False)

fig.show()

Como é possível verificar no gráfico acima, dos 48.895 acomodações listadas na plataforma, mais da metade (25.409 - representando 51,97%) são do tipo 'Entire home/apt', seguido por 22.326 do tipo 'Private Room' (45,66%), e apenas 1.160 são do tipo 'Shared room'(2,37%).

Agora vamos avaliar a distribuição das acomodações nos cinco distritos da cidade de Nova York - Manhattan, Brooklyn, Queens, Bronx e Staten Island.

In [39]:
contagem = airbnb['neighbourhood_group'].value_counts().values
distritos = airbnb['neighbourhood_group'].value_counts().index

fig = px.bar(
    x=contagem,
    y=distritos,
    orientation='h',
    title='Quantidade de Acomodações por Distritos',
    color=distritos,
    labels={'y': 'Distrito', 'x': 'Quantidade'},
    custom_data=[distritos, contagem])

fig.update_yaxes(autorange='reversed')

fig.update_layout(
    xaxis_title='Quantidade',
    yaxis_title='Distrito',
    showlegend=False
)

fig.update_traces(hovertemplate='Quantidade: %{customdata[1]}')

fig.show()


O gráfico acima demonstra que Manhattan mais de 85% das acomodações estão localizadas em Manhattan ou no Brooklyn. Staten Island é o distrito com a menor quantidade de imóveis, representando apenas 0,76% das acomodações listadas neste dataframe.

Além disso, conforme podemos visualizar abaixo, quando analisamos a distruibuição das acomodações por tipo de quarto, reafirmamos o dado de que, majoritariamente, os imóveis são do tipo Entire home/apt ou Private Room. Em Manhattan temos a maior concentração de apartamentos do tipo Entire home/apt, enquanto a maior número de apartamentos do tipo Private room está no Brooklyn.

In [40]:
pd.crosstab(airbnb['neighbourhood_group'],airbnb['room_type']).sort_values(by='Entire home/apt',ascending=False)

room_type,Entire home/apt,Private room,Shared room
neighbourhood_group,,,
Manhattan,13199,7982,480
Brooklyn,9559,10132,413
Queens,2096,3372,198
Bronx,379,652,60
Staten Island,176,188,9


Visto que já identificamos que a maior quantidade de acomodações está em Manhattan ou no Brooklyn, vamos verificar os bairros destes distritos que possuem a maior distruibuição de imóveis.

In [41]:
top10_bairros = airbnb.groupby(['neighbourhood','neighbourhood_group']).size().sort_values(ascending=False).reset_index(name='contagem')[:10]
top10_bairros

,neighbourhood,neighbourhood_group,contagem
0,Williamsburg,Brooklyn,3920
1,Bedford-Stuyvesant,Brooklyn,3714
2,Harlem,Manhattan,2658
3,Bushwick,Brooklyn,2465
4,Upper West Side,Manhattan,1971
5,Hell's Kitchen,Manhattan,1958
6,East Village,Manhattan,1853
7,Upper East Side,Manhattan,1798
8,Crown Heights,Brooklyn,1564
9,Midtown,Manhattan,1545


In [42]:
top10_bairros.groupby('neighbourhood_group')['contagem'].agg(['sum','count']).reset_index()

,neighbourhood_group,sum,count
0,Brooklyn,11663,4
1,Manhattan,11783,6


Quando avaliamos os top 10 bairros com mais acomodações listadas, é possível verifcar que embora as duas primeiras posições sejam ocupadas por bairros localizados no distrito do Brooklyn, Manhattan possui cerca cerca de 100 imóveis a mais neste ranking, representando um total de 6 bairros, dois a mais que no Brooklyn.

Nossas próximas análises estarão relacionadas aos preços das acomodações.

Conforme podemos verificar na tabela abaixo, o preço médio da diária é de US$ 152.72, com um desvio padrão de US$ 240.15. Este desvio padrão pode ser considerado elevado, indicando que há uma alta dispersão ou variabilidade dos preços em relação à média. Quando avaliamos os quartis, podemos perceber que a maior parte das acomodações possui diária pouco maior que a média e estão nos três primeiros quartis.

In [43]:
airbnb[['price']].describe().T

,count,mean,std,min,25%,50%,75%,max
price,48895.0,152.720687,240.15417,0.0,69.0,106.0,175.0,10000.0


In [44]:
print(f'Adicionalmente, identifica-se que {airbnb.query("price == 0").shape[0]} acomodações possuem preço igual a zero.')

Adicionalmente, identifica-se que 11 acomodações possuem preço igual a zero.


Vamos excluir do dataframe as acomodações com diárias zeradas, para que possamos visualizar melhor graficamente o preço das diárias médias, máximas e mínimas, por distrito, considerando o tipo de quarto.

In [45]:
airbnb = airbnb.query('price != 0')
precos = airbnb.groupby(['neighbourhood_group','room_type'])['price'].agg(['mean','median','min','max']).reset_index()

fig1 = px.bar(precos, x="neighbourhood_group", y="mean",
             color='room_type', barmode='group')

fig2 = px.bar(precos, x="neighbourhood_group", y="median",
             color='room_type', barmode='group')

fig3 = px.bar(precos, x="neighbourhood_group", y="max",
             color='room_type', barmode='group')

fig4 = px.bar(precos, x="neighbourhood_group", y="min",
             color='room_type', barmode='group')

fig = make_subplots(rows=2, cols=2, subplot_titles=('Média','Mediana','Preço Máximo', 'Preço Mínimo'))

for trace in fig1.data:
    fig.add_trace(trace, row=1, col=1)

for trace in fig2.data:
    fig.add_trace(trace, row=1, col=2)
    
for trace in fig3.data:
    fig.add_trace(trace, row=2, col=1)

for trace in fig4.data:
    fig.add_trace(trace, row=2, col=2)

fig.update_layout(height=700, width=1100, title='Preço das Diárias por Tipo de Quarto e Distrito',showlegend=False)
fig.update_xaxes(tickangle=-90)
fig.show()

In [46]:
airbnb.groupby('neighbourhood_group')['price'].agg(['mean','median','std','min','max']).reset_index()

,neighbourhood_group,mean,median,std,min,max
0,Bronx,87.577064,65.0,106.725371,10,2500
1,Brooklyn,124.438915,90.0,186.896837,10,10000
2,Manhattan,196.884903,150.0,291.386838,10,10000
3,Queens,99.517649,75.0,167.102155,10,10000
4,Staten Island,114.812332,75.0,277.620403,13,5000


In [47]:
precos['diff'] = precos['mean'] - precos['median']
precos

,neighbourhood_group,room_type,mean,median,min,max,diff
0,Bronx,Entire home/apt,127.506596,100.0,28,1000,27.506596
1,Bronx,Private room,66.890937,54.0,10,2500,12.890937
2,Bronx,Shared room,59.800000,40.0,20,800,19.800000
3,Brooklyn,Entire home/apt,178.346202,145.0,10,10000,33.346202
4,Brooklyn,Private room,76.545428,65.0,10,7500,11.545428
5,Brooklyn,Shared room,50.773723,36.0,15,725,14.773723
6,Manhattan,Entire home/apt,249.257994,191.0,10,10000,58.257994
7,Manhattan,Private room,116.776622,90.0,10,9999,26.776622
8,Manhattan,Shared room,88.977083,69.0,10,1000,19.977083
9,Queens,Entire home/apt,147.050573,120.0,10,2600,27.050573


Com base nos gráficos e nas tabelas acima, cabe citar os seguintes pontos:
* Brooklyn, Manhattan e Queens tem o mesmo preço máximo;
* Com exceção de Staten Island, demais distritos possuem o mesmo preço mínimo;
* O maior preço médio de diárias está em Manhattan, e o menor, no Bronx;
* No Bronx e Queens o tipo de quarto mais caro é o Private Room (e não o Entire home/apt conforme seria esperado);
* A diária mínima para Staten Island é a maior quando comparada aos demais distrito;
* A mediana - número central de uma lista de dados organizados de forma crescente, é relativamente inferior a média independentemente do tipo de quarto e distrito. A maior diferença é percebida nos quartos do tipo Entire home/apt em Manhattan e Staten Island (aproximadamente US$ 58 e US$ 74, respectivamente).

Agora, vamos verificar onde estão localizados as acomodações com as diárias mais caras, e qual o respectivo tipo de quarto.

In [48]:
airbnb[['id','neighbourhood_group','neighbourhood','room_type','price']].sort_values(by='price',ascending=False)[:10]

,id,neighbourhood_group,neighbourhood,room_type,price
29238,22436899,Manhattan,Upper West Side,Entire home/apt,10000
17692,13894339,Brooklyn,Greenpoint,Entire home/apt,10000
9151,7003697,Queens,Astoria,Private room,10000
12342,9528920,Manhattan,Lower East Side,Private room,9999
40433,31340283,Manhattan,Lower East Side,Entire home/apt,9999
6530,4737930,Manhattan,East Harlem,Entire home/apt,9999
30268,23377410,Manhattan,Tribeca,Entire home/apt,8500
4377,2953058,Brooklyn,Clinton Hill,Entire home/apt,8000
29662,22779726,Manhattan,Upper East Side,Entire home/apt,7703
42523,33007610,Manhattan,Battery Park City,Entire home/apt,7500


As acomodações mais caras estão no Queens, Brooklyn e Manhattan, sendo que este último distrito, possui a maior quantidade de imóveis dentre os mais caros. Chama a atenção que temos quartos do tipo Private Room entre os mais caros.

Agora vamos verificar as acomodações com as maiores diárias em cada bairro.

In [49]:
neighbourhood_group = airbnb['neighbourhood_group'].unique()
for i in neighbourhood_group:
    bairro = airbnb[airbnb.neighbourhood_group == i].reset_index()
    bairro = bairro.groupby(['neighbourhood','room_type'])['price'].max().reset_index(name='max_price').sort_values(by='max_price',ascending=False)[:10]
    print(i)
    display(bairro)

Brooklyn


,neighbourhood,room_type,max_price
81,Greenpoint,Entire home/apt,10000
34,Clinton Hill,Entire home/apt,8000
58,East Flatbush,Private room,7500
5,Bedford-Stuyvesant,Entire home/apt,5000
6,Bedford-Stuyvesant,Private room,5000
47,Cypress Hills,Entire home/apt,5000
120,Williamsburg,Entire home/apt,4500
2,Bay Ridge,Entire home/apt,4200
98,Prospect Heights,Entire home/apt,4000
20,Brighton Beach,Private room,3000


Manhattan


,neighbourhood,room_type,max_price
81,Upper West Side,Entire home/apt,10000
11,East Harlem,Entire home/apt,9999
44,Lower East Side,Private room,9999
43,Lower East Side,Entire home/apt,9999
74,Tribeca,Entire home/apt,8500
78,Upper East Side,Entire home/apt,7703
0,Battery Park City,Entire home/apt,7500
3,Chelsea,Entire home/apt,6800
82,Upper West Side,Private room,6500
25,Greenwich Village,Entire home/apt,6000


Queens


,neighbourhood,room_type,max_price
4,Astoria,Private room,10000
9,Bayside,Entire home/apt,2600
47,Forest Hills,Entire home/apt,2350
80,Long Island City,Entire home/apt,2000
5,Astoria,Shared room,1800
0,Arverne,Entire home/apt,1500
3,Astoria,Entire home/apt,1500
44,Flushing,Entire home/apt,1500
82,Long Island City,Shared room,1250
19,Briarwood,Entire home/apt,1000


Staten Island


,neighbourhood,room_type,max_price
49,Randall Manor,Entire home/apt,5000
47,Prince's Bay,Entire home/apt,1250
63,St. George,Entire home/apt,1000
20,Fort Wadsworth,Entire home/apt,800
78,Woodrow,Entire home/apt,700
2,Arrochar,Entire home/apt,625
65,Stapleton,Entire home/apt,450
67,Todt Hill,Entire home/apt,429
56,Shore Acres,Entire home/apt,300
57,Shore Acres,Private room,300


Bronx


,neighbourhood,room_type,max_price
85,Riverdale,Private room,2500
12,City Island,Entire home/apt,1000
86,Riverdale,Shared room,800
46,Longwood,Private room,680
112,Westchester Square,Entire home/apt,670
114,Williamsbridge,Entire home/apt,500
60,Mott Haven,Private room,500
27,Eastchester,Entire home/apt,475
99,Unionport,Entire home/apt,450
78,Pelham Gardens,Entire home/apt,450


Esperava-se que somente acomodações do tipo Private room e Entire home/apt estivessem entre as mais caras, no entanto, é possível verificar que no Queens e no Bronx temos acomodações do tipo Shared room entre as mais caras.

Vamos visualizar em um mapa da cidade de Nova York onde estão localizados as 50 acomodações mais caras.

In [50]:
# Criando um dataframe com os 50 apartamentos mais caros
top_50 = airbnb.sort_values(by='price',ascending=False)[:50]

# Criando o mapa
mapa = folium.Map(location=[40.7128, -74.0060], zoom_start=11)  

# Adicionando marcadores para cada apartamento
for i in range(0,len(top_50)):
   folium.Marker(
      location=[top_50.iloc[i]['latitude'], top_50.iloc[i]['longitude']],
      popup=f"Name: {top_50.iloc[i]['name']} | District: {top_50.iloc[i]['neighbourhood_group']} | Price: US$ {top_50.iloc[i]['price']:,.2f}",
   ).add_to(mapa)

# Exibindo o mapa
mapa

Nossas próximas análises estatarão relacionadas ao número mínimo de noites.

Conforme podemos verificar abaixo, de forma geral, a média do mínimo de noites é 7, com um desvio padrão relativamente alto de 20 noites. Além disso, chama a atenção o máximo de noites relacionado na base, que é de 1250 noites (aproximadamente 42 meses).

In [51]:
airbnb[['minimum_nights']].describe().T

,count,mean,std,min,25%,50%,75%,max
minimum_nights,48884.0,7.029887,20.512224,1.0,1.0,3.0,5.0,1250.0


Vamos separar o mínimo de noite em alguns intervalos:

In [52]:
intervalos = [0, 10, 30, 90, 180, 365, float('inf')]
labels = ['até 10 noites', 'maior que 10 até 30 noites', 'mais de 30 até 90 noites','maior que 90 até 180 noites', 'maior que 180 até 365 noites', 'maior que 365 noites']
airbnb['categoria'] = pd.cut(airbnb['minimum_nights'], bins=intervalos, labels=labels)
nights = airbnb.groupby('categoria')['minimum_nights'].count().reset_index(name='qtd')
nights['%_total'] = nights['qtd']/48884
nights

,categoria,qtd,%_total
0,até 10 noites,42246,0.864209
1,maior que 10 até 30 noites,5891,0.120510
2,mais de 30 até 90 noites,550,0.011251
3,maior que 90 até 180 noites,116,0.002373
4,maior que 180 até 365 noites,67,0.001371
5,maior que 365 noites,14,0.000286


In [53]:
pivot = airbnb.groupby(['categoria','room_type'])['minimum_nights'].count().reset_index()
pivot = pd.pivot_table(pivot,values='minimum_nights',index='categoria',columns=['room_type']).reset_index()
pivot

room_type,categoria,Entire home/apt,Private room,Shared room
0,até 10 noites,21094.0,20143.0,1009.0
1,maior que 10 até 30 noites,3809.0,1950.0,132.0
2,mais de 30 até 90 noites,377.0,164.0,9.0
3,maior que 90 até 180 noites,72.0,37.0,7.0
4,maior que 180 até 365 noites,46.0,21.0,0.0
5,maior que 365 noites,9.0,4.0,1.0


Mais de 98% das acomodações listadas exigem um mínimo de noites de até 30 dias, sendo um percentual superior a 86% para apartamentos com mínimo de noites de até 10 dias. Quando analisamos a distribuiçao do diferentes room_types nas faixas de mínimo de noites, também identificamos uma elevada concentração de acomodações na faixa de até 30 dias.

Agora vamos avaliar a média, mediana, moda, mínimo e máximo de noites por distrito.

In [54]:
airbnb.groupby('neighbourhood_group')['minimum_nights'].agg(['mean','median',pd.Series.mode,'min','max']).reset_index()

,neighbourhood_group,mean,median,mode,min,max
0,Bronx,4.563303,2.0,1,1,365
1,Brooklyn,6.055486,3.0,2,1,999
2,Manhattan,8.579409,3.0,1,1,1250
3,Queens,5.181433,2.0,1,1,500
4,Staten Island,4.831099,2.0,2,1,365


Quando olhamos para a média, o Bronx possui a menor média de noites, enquanto Manhattan, a maior. 

Com relação à Mediana, verifica-se que esta é igual a 2 noites tanto no Brooklyn como em Manhattan, e para os demais distritos, igual a 2.

Já a Moda (valor que mais se repete num conjunto de valores), é igual a 2 noites no Brooklyn e Staten Island, e a 1 noite nos demais distritos.

Novamente o máximo de noites chama a atenção, e podemos verificar que a acomodação com o maior número mínimo de noites está localizado em Manhattan. Adicionalmento, no Brooklyn também temos um imóvel com mínimo de noites superior a 1 ano (999 noites - aproximadamente 33 meses), assim como no Queens (500 noites - aproximadamente 17 meses).

Abaixo podemos visualizar quais são estas acomodações com número mínimo de noite superior a 365 dias e onde estão localizados. Dos 48.895 acomodações listadas na base em análise, apenas 14 possuem o nínimo de noites superior a 365 dias (0,03%). Além disso, é possível verificar que Staten Island e Bronx não possuem imóveis onde o mínimo de noites é maior que um ano.

In [55]:
max_nights = airbnb.query('minimum_nights > 365').sort_values(by='minimum_nights',ascending=False).reset_index()
max_nights[['name','neighbourhood_group','room_type','price','minimum_nights']]

,name,neighbourhood_group,room_type,price,minimum_nights
0,Prime W. Village location 1 bdrm,Manhattan,Entire home/apt,180,1250
1,NaN,Manhattan,Entire home/apt,400,1000
2,Historic Designer 2 Bed. Apartment,Manhattan,Entire home/apt,99,999
3,Beautiful place in Brooklyn! #2,Brooklyn,Private room,79,999
4,Shared Studio (females only),Manhattan,Shared room,110,999
5,Beautiful Fully Furnished 1 bed/bth,Queens,Entire home/apt,134,500
6,Wonderful Large 1 bedroom,Manhattan,Entire home/apt,75,500
7,Zen Room in Crown Heights Brooklyn,Brooklyn,Private room,50,500
8,Peaceful apartment close to F/G,Brooklyn,Private room,45,500
9,Williamsburg Apartment,Brooklyn,Entire home/apt,140,500


Agora vamos verificar qual a média de preços para as acomodações nas diferentes faixas de mínimo de noites.

In [56]:
airbnb.groupby('categoria')['price'].mean().reset_index()

,categoria,price
0,até 10 noites,149.540809
1,maior que 10 até 30 noites,164.886437
2,mais de 30 até 90 noites,194.176364
3,maior que 90 até 180 noites,473.060345
4,maior que 180 até 365 noites,221.388060
5,maior que 365 noites,137.571429


Conforme verifica-se acima, as acomodações com mínimo de diárias superior a 365 apresentam a menor média de preços. Chama a atenção que os apartamentos com mínimo de diárias maior que 90 até 180 noites, apresenta média maior que o dobro das demais faixas.

Por fim, vamos olhar para a quantidade de avaliações dos imóveis listados na base. 

Quando olhamos apenas para as acomodações que mais receberam avaliações, é possível identificar que estão localizados no Queens, Manhattan e Brooklyn. Não temos imóveis no Bronx e Staten Island entre os que mais receberam avaliações.

In [57]:
airbnb[['name','neighbourhood_group','room_type','number_of_reviews','reviews_per_month']].sort_values(by='number_of_reviews',ascending=False)[:10]

,name,neighbourhood_group,room_type,number_of_reviews,reviews_per_month
11759,Room near JFK Queen Bed,Queens,Private room,629,14.58
2031,Great Bedroom in Manhattan,Manhattan,Private room,607,7.75
2030,Beautiful Bedroom in Manhattan,Manhattan,Private room,597,7.72
2015,Private Bedroom in Manhattan,Manhattan,Private room,594,7.57
13495,Room Near JFK Twin Beds,Queens,Private room,576,13.40
10623,Steps away from Laguardia airport,Queens,Private room,543,11.59
1879,Manhattan Lux Loft.Like.Love.Lots.Look !,Manhattan,Private room,540,6.95
20403,Cozy Room Family Home LGA Airport NO CLEANING FEE,Queens,Private room,510,16.22
4870,Private brownstone studio Brooklyn,Brooklyn,Entire home/apt,488,8.14
471,LG Private Room/Family Friendly,Brooklyn,Private room,480,6.70


Abaixo, relacionamos as acomodações por distrito que mais receberam avaliações e quantas avaliações receberam por mês.

In [58]:
idx = airbnb.groupby('neighbourhood_group')['number_of_reviews'].idxmax()
top_avaliados = airbnb.loc[idx, ['neighbourhood_group', 'name', 'number_of_reviews','reviews_per_month']].sort_values(by='number_of_reviews',ascending=False)
top_avaliados

,neighbourhood_group,name,number_of_reviews,reviews_per_month
11759,Queens,Room near JFK Queen Bed,629,14.58
2031,Manhattan,Great Bedroom in Manhattan,607,7.75
4870,Brooklyn,Private brownstone studio Brooklyn,488,8.14
256,Staten Island,D Private Che@p Room 2 Explore NYC,333,3.19
5260,Bronx,Cozy Private Bedroom,321,5.46


In [59]:
bottom_avaliados = airbnb[airbnb.number_of_reviews == 0]
print(f'Identificamos que um total de {bottom_avaliados.shape[0]} acomodações não receberam nenhuma avaliação.')

Identificamos que um total de 10051 acomodações não receberam nenhuma avaliação.


Agora, vamos verificar qual o percentual de acomodações por distrito que não receberam nenhuma avaliação.

In [60]:
# Criando novo dataframe com o total de imóveis por distrito que não receberam nenhuma avaliação
bottom_avaliados = bottom_avaliados.groupby('neighbourhood_group')['id'].count().reset_index(name='count').sort_values(by='count',ascending=False)

# Criando novo dataframe com o total de imóveis por distrito 
total_aptos = airbnb.groupby('neighbourhood_group')['id'].count().reset_index(name='total_count')

# Realizando o merge dos dataframe de imóveis sem avaliação + total de apartamentos
bottom_avaliados = bottom_avaliados.merge(total_aptos, how='left', on='neighbourhood_group')

# Adicionando nova coluna ao dataframe com o percentual de imóveis por distrito que não recebeu nenhuma avaliação
bottom_avaliados['%_no_review'] = round(((bottom_avaliados['count'] / bottom_avaliados['total_count'])*100),2)

# Visualizando o dataframe criado
bottom_avaliados

,neighbourhood_group,count,total_count,%_no_review
0,Manhattan,5028,21660,23.21
1,Brooklyn,3657,20095,18.20
2,Queens,1092,5666,19.27
3,Bronx,215,1090,19.72
4,Staten Island,59,373,15.82


Para finalizar, vamos visualizar as informações acima graficamente.

In [61]:
distrito = bottom_avaliados['neighbourhood_group'].value_counts().index
total_ap = bottom_avaliados['total_count'].value_counts().index
sem_aval = bottom_avaliados['count'].value_counts().index
percent = bottom_avaliados['%_no_review'].value_counts().index

# Criando o gráfico total
barra_total = go.Bar(x=distrito, y=total_ap,
               name='Total')
           
# Criando o gráfico sem avaliação
barra_zero = go.Bar(x=distrito, y=sem_aval,
               name='Sem Avaliação')

# Adicionando percentuais nas barras 'sem avaliação'
annotations= []
for i in range(len(sem_aval)):
    annotation = dict(
        x=i + 0.25,
        y=sem_aval[i] + 50,
        text=f'{percent[i]}%',
        showarrow=True,
        arrowhead=3,
        ax=0
    )
    annotations.append(annotation)
               
# Definindo o layout
layout = go.Layout(title='Total de Quartos por Distrito x Quartos sem Avaliação (%)',
                   # Configurações do título
                   titlefont= {'family':'Arial', 'size':25, 'color':'#0e78a3'},
                   xaxis={'title':'Distrito'},
                   yaxis={'title':'Quantidade de Quartos'},
                   barmode='group',
                   height=500, width=1000,
                   annotations=annotations)

# Criando a figura
fig = go.Figure(data=[barra_total,barra_zero], layout=layout)

# Plotando o gráfico
py.iplot(fig)

#### **CONCLUSÃO**
Nova York é uma metrópole globalmente conhecida como centro de negócios e destino turístico vibrante. Está dividida em cinco distritos, sendo eles: Manhattan, Brooklyn, Queens, Bronx e Staten Island.

Manhattan e Brooklyn são as regiões mais populares e turísticas. Manhattan, mesmo sendo menor em área, possui a maior densidade demográfica, inúmeras atrações turísticas, além de ser o epicentro financeiro da cidade. 

Enquanto isso, o Brooklyn se destaca como o distrito mais populoso e um verdadeiro caldeirão cultural, com bairros que respiram história e arte.

O Queens, o maior distrito em extensão, é pouco explorado por turistas, mas está estrategicamente localizado próximo aos principais aeroportos da cidade. Nos últimos anos, passou por um desenvolvimento significativo, com foco na construção e renovação de prédios residenciais, elevando assim os custos de moradia.

O Bronx, embora seja pouco visitado por turistas pois, infelizmente, muitas vezes é associado à pobreza e criminalidade, possui alguns atrativos relavantes, como o Jardim Zoológico do Bronx e o Yankee Stadium, palco de grandes eventos esportivos.

Por fim, Staten Island, é o distrito mais meridional e menos conhecido, oferecendo um refúgio tranquilo e uma atmosfera suburbana, contrastando fortemente com a agitação do resto da cidade.

Nova York é conhecida por seus preços elevados, desde diárias de hotéis até o custo de vida em geral. Quando olhamos para a plataforma AirBnB - plataforma online que permite às pessoas alugar acomodações alternativas aos hotéis tradicionais, onde os anfitriões disponibilizam desde quartos individuais a casas inteiras para viajantes, as acomodações listadas na cidade de Nova York corroboram com as características gerais da cidade, quanto aos custos elevados, distritos mais caros e populares. Após EDA realizada, os principais insights foram:

* Mais da metade das acomodações listadas são do tipo Entire home/apt inteiros, sendo que somado aos apartamentos do tipo Private, somam mais de 97% dos apartamentos analisados. Pouco mais de 2% dos apartamentos são do tipo Shared room. Esta distribuição dos tipos de apartamento pode refletir tanto a preferência como o orçamento dos visitantes - a diária de apartamentos inteiros ou quartos privados costuma ser mais cara;

* A popularidade dos distritos de Manhattan e Brooklyn reflete na quantidade de acomodações ofertadas nestas regiões - mais de 85% dos imóveis relacionados no dataset;

* O Queens possui cerca de 12% das acomodações listadas, o que pode estar somente relacionado somente a sua maior extensão geográfica ou também ao desenvolvimento observado nos últimos anos;

* Um percentual superior a 98% dos apartamentos exige um mínimo de noites de até 30 dias, independetemente do room_type;

* Manhattan e Brooklyn, em linha com sua popularidade, possuem as maiores médias de diária. Quando olhamos para o valor máximo de diária, além destes dois distrito, também no Queens observa-se o valor de US$ 10.000. O preço das diárias de uma forma geral reforçam a fama que a cidade de Nova York tem de possuir preços elevados.

* Chama a atenção a oferta de apartamentos com mínimo de noites superior a 30 dias, o que pode indicar que os imóveis listados na plataforma não são direcionado somente a turistas, mas também a outros públicos, que precisam ficam um prazo superior na cidade por um prazo maior. Os alugueis por mínimo de noite superior a 365 dias tendem a possuir a diária mais barata.

* Verfica-se um percentual elevado de acomodações sem avaliação, pode-se supor que são apartamentos listados mais recentemente na plataforma e por este motivo ainda não tiveram avaliações.
